In [4]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy, onehotbatch, crossentropy 
using MLDatasets
using Random
using Statistics
using MLJBase
using Printf
using BSON: @load # for load weights
using Plots
using DelimitedFiles
import Evolutionary.initial_population
import Evolutionary.EvolutionaryObjective
using Zygote
import Base: copy
using StableRNGs

random seed for Evolutionary

In [5]:
Random.seed!(63456345)


TaskLocalRNG()

Read in and balance data

In [6]:
rawdata = readdlm("data/test2.csv",',',skipstart=1)';
one = 0
zero = 0
for i in rawdata[20,:]
    if i == 1
        one = one + 1
    else
        zero = zero + 1
    end
end

difference = one - zero

function newrand()
    randCol = zeros(0)
    push!(randCol, rand(10.0:99.0))
    push!(randCol, rand(5.3:40.2))
    push!(randCol, rand(1.1:15.1))
    push!(randCol, rand(0.1:8.1))
    push!(randCol, rand(1.1:15.1))
    push!(randCol, rand(30.1:60.1))
    push!(randCol, rand(0.1:0.9))
    push!(randCol, rand(0.1:4.1))
    push!(randCol, rand(20.1:50.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(45.1:99.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(0.1:3.1))
    push!(randCol, rand(0.1:3.1))
    push!(randCol, rand(0.1:5.1))
    if rand(0:1) == 0
        push!(randCol, 0.0)
    else
        push!(randCol, 1.0)
    end
    return randCol
end

for i = 1:size(rawdata,2)
    if rawdata[20,i] == 0 && difference > 0
        rawdata = hcat(rawdata, rawdata[:,i])
        rawdata = hcat(rawdata, newrand())

        difference = difference - 1
    end
end
filldata = rawdata[ :, shuffle(1:end)];

x = filldata[1:19, :]
y = filldata[20, :];

x_train = x[:,1:floor(Int, size(x,2)*0.7)]
y_train = y[1:floor(Int, size(x,2)*0.7)]
x_test = x[:,floor(Int, size(x,2)*0.7)+1:end]
y_test = y[floor(Int, size(x,2)*0.7)+1:end];


zip feature & label together using zip instead of batch for Evolutionary function

In [7]:
train_data = [ (x, onehot(l, unique(y_train))) for (x, l) in zip(eachcol(x_train), y_train)]
test_data = [ (x, onehot(l, unique(y_test))) for (x, l) in zip(eachcol(x_test), y_test)];

In [8]:
accuracy(model,x,y) = mean(onecold(model(x)) .== onecold(y))
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)

loss(model) = (x,y)->logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

the fitness function of evolutionary function

In [9]:
fitness(m) = loss(train_data, m)

fitness (generic function with 1 method)

flat the weight and bias for evolutionary function to evolutionary

In [10]:
function uniform_mlp(m1::T, m2::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 = UX(θ1,θ2; rng=rng)
    return re1(c1), re2(c2)
end

function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}  
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ; rng=rng)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

the initial_population function of evolutionary function

In [11]:
function initial_population(method::M, individual::Chain;
    rng::Random.AbstractRNG=Random.default_rng(),
    kwargs...) where {M<:Evolutionary.AbstractOptimizer}
θ, re = Flux.destructure(individual);
[re(randn(rng, length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 8 methods)

Overload some function for evolutionary function

In [13]:
function EvolutionaryObjective(f, x::Chain; eval::Symbol = :serial)
    fval = f(x)
    EvolutionaryObjective{typeof(f),typeof(fval),typeof(x),Val{eval}}(f, fval, deepcopy(x), 0)
end

copy(ch::Chain) = deepcopy(ch)

copy (generic function with 176 methods)

In [21]:
# the iterations is performed later
opts = Evolutionary.Options(iterations=100, successive_f_tol=25, rng=StableRNG(42))
algo = GA(
        selection = rouletteinv,
        mutation =  gaussian_mlp(),
        crossover = uniform_mlp,
        mutationRate = 0.2,
        crossoverRate = 0.9,
        populationSize = 1600,
        ε = 0.0003
    )

GA[P=1600,x=0.9,μ=0.2,ɛ=0.0003]

samilar code as backpropagation but change the training part using evolutionary.ji

In [28]:
# epochs = 2000 # the number of epochs
# # change population_size will increase training time, but may increase the accuracy
# save_resultTest = zeros(epochs);
# save_resultTrain = zeros(epochs);

# model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))
# # load the models saved from HW1, the softmax is removed because it does not need to train

# for i in 1:10
#     # change the model
#     # load the models saved from HW1
#     loadpath = string("models/model",i,".bson")
#     @load loadpath weights
#     Flux.loadparams!(model, weights)
#     # the models are started with the same hyperparameters
#     res = Evolutionary.optimize(fitness, model, algo, opts)
#     evomodel = Evolutionary.minimizer(res)
#     # training the model
#     for j in 1:epochs
#         res = Evolutionary.optimize(fitness, model, algo, opts)
#         model= Evolutionary.minimizer(res)
#         if j%(epochs/5) == 0
#         @printf("Loss in expirment %d epoch: %d in test data is %f\n",i, j, loss(test_data,model))
#         end
#         save_resultTest[j] = save_resultTest[j] + accuracy(test_data, model)
#         save_resultTrain[j] = save_resultTrain[j] + accuracy(train_data, model)
#     end
# end

In [22]:
#model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))

res = Evolutionary.optimize(fitness, model, algo, opts)


 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer:  Chain(Dense(19, 38, σ), Dense(38, 2))
    Minimum:    0.6703265664281358
    Iterations: 100

 * Found with
    Algorithm: GA[P=1600,x=0.9,μ=0.2,ɛ=0.0003]

 * Convergence measures
    |f(x) - f(x')| = 0.012819815334979778 ≰ 1.0e-12

 * Work counters
    Seconds run:   397.12 (vs limit Inf)
    Iterations:    100
    f(x) calls:    161600


In [23]:
model= Evolutionary.minimizer(res)


Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 6.797 KiB.

In [35]:
#model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))
res = Evolutionary.optimize(fitness, model, algo, opts)
model= Evolutionary.minimizer(res)

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 6.797 KiB.

In [30]:
# save_resultTest = save_resultTest ./ 10;
# save_resultTrain = save_resultTrain ./ 10;

# plot(log.(1:epochs), save_resultTest,label="Test")
# plot!(log.(1:epochs), save_resultTrain,label = "Train", title = "Accuracy", legend = :outertopleft)

In [24]:
@info "MLP" loss=loss(train_data, model) accuracy = accuracy(train_data, model)

┌ Info: MLP
│   loss = 0.6703265664281358
│   accuracy = 0.595821325648415
└ @ Main In[24]:1


In [18]:
simplified_x_train_result = []
for i  = 1:size(x_train,2)
    if softmax(model(x_train[:,i]))[1]>0.5
        push!(simplified_x_train_result, 0)
    else
        push!(simplified_x_train_result, 1)
    end
end
simplified_y_train = []
for i  = 1:size(x_train,2)
    if y_train[i] == 0
        push!(simplified_y_train, 0)
    else
        push!(simplified_y_train, 1)
    end
end

# ConfusionMatrix for the training data
print("ConfusionMatrix for the training data\n")
ConfusionMatrix()(simplified_x_train_result, simplified_y_train)

ConfusionMatrix for the training data


┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\xkzmx\.julia\packages\MLJBase\pCiRR\src\measures\confusion_matrix.jl:112


              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     391     │     538     │
├─────────────┼─────────────┼─────────────┤
│      1      │     298     │     161     │
└─────────────┴─────────────┴─────────────┘


In [19]:
simplified_x_test_result = []
for i  = 1:size(x_test,2) 
    if softmax(model(x_test[:,i]))[1]>0.5
        push!(simplified_x_test_result, 0)
    else
        push!(simplified_x_test_result, 1)
    end
end
simplified_y_test = []
for i  = 1:size(x_test,2) 
    if y_test[i] == 0
        push!(simplified_y_test, 0)
    else
        push!(simplified_y_test, 1)
    end
end

# ConfusionMatrix for the test data
print("ConfusionMatrix for the test data\n")
ConfusionMatrix()(simplified_x_test_result, simplified_y_test)

ConfusionMatrix for the test data


┌ Warning: The classes are un-ordered,
│ using: negative='0' and positive='1'.
│ To suppress this warning, consider coercing to OrderedFactor.
└ @ MLJBase C:\Users\xkzmx\.julia\packages\MLJBase\pCiRR\src\measures\confusion_matrix.jl:112


              ┌───────────────────────────┐
              │       Ground Truth        │
┌─────────────┼─────────────┬─────────────┤
│  Predicted  │      0      │      1      │
├─────────────┼─────────────┼─────────────┤
│      0      │     171     │     227     │
├─────────────┼─────────────┼─────────────┤
│      1      │     135     │     63      │
└─────────────┴─────────────┴─────────────┘
